In [10]:
import numpy as np

## Problem data (IEEE14)

In [27]:
n = 14
Pmin = np.array([-21.7,-94.2,-47.8,-7.6,-11.2,-29.5,-9.,-3.5,-6.1,-13.5,-14.9,0.,0.,0.])
Pmax = np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,20.,50.,10.])
Pren = np.array([0.,0.,0.,14.9,9.,10.,18.5,2.,7.,5.,5.,0.,0.,0.])
a = np.array([4.,1.,2.,10.,8.,4.,9.,15.,18.,7.,6.,7.,2.,8.])
b = np.array([130.,120.,135.,125.,140.,145.,150.,135.,140.,125.,120.,25.,10.,20.])
Q = np.diag(a)

In [36]:
from cvxpy import *
x = Variable(n)
obj = quad_form(x, Q) + b*x
constraints = [sum_entries(x) + sum_entries(Pren) == 0, 
                Pmin <= x, x <= Pmax]
prob = Problem(Minimize(obj), constraints)
prob.solve()
print(prob.value)
print("Schedule: ")
print(x.value)
print("Price: ",constraints[0].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(x.value * constraints[0].dual_value)

-8052.111736270549
Schedule: 
[[ -8.26637071]
 [-28.0654519 ]
 [-17.78272935]
 [ -3.05656276]
 [ -4.7581915 ]
 [-10.14136791]
 [ -4.78504585]
 [ -2.37096286]
 [ -2.11476871]
 [ -4.36651219]
 [ -4.6775958 ]
 [  2.77640618]
 [ 13.46729717]
 [  2.74185618]]
Price:  -63.86829931198621
Payments and revenues under uniform pricing scheme: 
[[  527.9590386 ]
 [ 1792.49268229]
 [ 1135.75268055]
 [  195.21746493]
 [  303.89759898]
 [  647.71192141]
 [  305.61274051]
 [  151.42936558]
 [  135.06668105]
 [  278.88170777]
 [  298.75008835]
 [ -177.32434112]
 [ -860.13336684]
 [ -175.1176911 ]]
